In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
import put101 as p
import dotenv
import os
import MetaTrader5 as mt5
from MetaTrader5 import AccountInfo, TerminalInfo
from datetime import datetime
import logging

import importlib
importlib.reload(mt5)
importlib.reload(p)



<module 'put101' from 'C:\\Users\\puche\\code\\nautilus\\put101\\__init__.py'>

# 1. Connect to MT5

In [67]:

# load data from the MetaTrader 5 terminal into parquet files for further backtesting
if not dotenv.load_dotenv():
    logging.log(logging.INFO, "No .env file found")

MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.environ["CATALOG_PATH"]

loader_config = p.MTLoginConfig(server=MT5_SERVER, login=MT5_LOGIN, password=MT5_PASSWORD)

venue_EIGHTCAP = "SIM"
loader = p.MT5Loader(data_path=DATA_PATH, catalog_path=CATALOG_PATH, config=loader_config, venue=venue_EIGHTCAP)



In [68]:
loader.init()

True

In [77]:
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.data import BarType, BarSpecification, BarAggregation
# load a couple of symbols into the catalog using the loader
symbols = ['EURUSD', 'GBPUSD', 'USDJPY']
timeframes = [mt5.TIMEFRAME_M15, mt5.TIMEFRAME_M5, mt5.TIMEFRAME_M3, mt5.TIMEFRAME_H1, mt5.TIMEFRAME_H4, mt5.TIMEFRAME_D1]
start_date = datetime(2000, 1, 1)




for symbol in symbols:
    for timeframe in timeframes:
        delete_load(symbol=symbol, timeframe=timeframe, start_date=start_date)



INFO: Deleting instrument at: catalog\EURUSD.SIM-15M-LAST-EXTERNAL
ERROR: [WinError 2] Das System kann die angegebene Datei nicht finden: 'catalog\\EURUSD.SIM-15M-LAST-EXTERNAL'
INFO: Loaded EURUSD.SIM with bartype 15-MINUTE-MID into catalog
INFO: Deleting instrument at: catalog\EURUSD.SIM-5M-LAST-EXTERNAL
ERROR: [WinError 2] Das System kann die angegebene Datei nicht finden: 'catalog\\EURUSD.SIM-5M-LAST-EXTERNAL'
INFO: Loaded EURUSD.SIM with bartype 5-MINUTE-MID into catalog
INFO: Deleting instrument at: catalog\EURUSD.SIM-3M-LAST-EXTERNAL
ERROR: [WinError 2] Das System kann die angegebene Datei nicht finden: 'catalog\\EURUSD.SIM-3M-LAST-EXTERNAL'
INFO: Loaded EURUSD.SIM with bartype 3-MINUTE-MID into catalog
INFO: Deleting instrument at: catalog\EURUSD.SIM-16385M-LAST-EXTERNAL
ERROR: [WinError 2] Das System kann die angegebene Datei nicht finden: 'catalog\\EURUSD.SIM-16385M-LAST-EXTERNAL'
INFO: Loaded EURUSD.SIM with bartype 16385-MINUTE-MID into catalog
INFO: Deleting instrument at:

In [3]:
def delete_load(symbol, timeframe, start_date):
    loader.delete_catalog_parquet_files(symbol=symbol, timeframe=timeframe)
    loader.load(symbol=symbol, timeframe=timeframe, start=start_date)
    bar_type = loader.get_bar_type(symbol=symbol, timeframe=timeframe)
    instrument = loader.get_instrument_FOREX(symbol=symbol)
    print(f"INFO: Loaded {instrument.id} with bartype {bar_type.spec} into catalog")

In [4]:
delete_load("EURUSD", 15, datetime(2000, 1, 3))

NameError: name 'datetime' is not defined

INFO: Loaded EURUSD.SIM with bartype 16385-MINUTE-MID into catalog
